In [ ]:
import cv2 as cv
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import normalize
from sklearn.metrics import silhouette_samples, silhouette_score
import PIL.ImageColor as ImageColor
from sklearn.cluster import KMeans

In [ ]:
# data for clustering algorithm
data_index = ["H-15", "A-15", "B-11", "A-7", "F-9", "G-3", "G-11", "H-7", "I-13"]
path_cores = "TMA_cores_M06_M07_panels/M06/Cores/"
path_mxIF = "Texts_small_coregistered/"

In [ ]:
data_cores = [cv.imread(path_cores + index + ".png") for index in data_index]
data_mxIF = [pd.read_csv(path_mxIF + index + ".csv") for index in data_index]

In [ ]:
BUFFER = 50
BATCH = 64
CELL_SIZE = (32, 32)
MXIF_FEATURES = ["Nucleus PD1 (PPD520) Mean (Normalized Counts, Total Weighting)",
                 "Nucleus PD1 (PPD520) Max (Normalized Counts, Total Weighting)",
                 "Nucleus PD1 (PPD520) Std Dev (Normalized Counts, Total Weighting)",
                 "Nucleus FOXP3 (PPD540) Mean (Normalized Counts, Total Weighting)",
                 "Nucleus FOXP3 (PPD540) Max (Normalized Counts, Total Weighting)",
                 "Nucleus FOXP3 (PPD540) Std Dev (Normalized Counts, Total Weighting)",
                 "Nucleus CD20 (PPD620) Mean (Normalized Counts, Total Weighting)",
                 "Nucleus CD20 (PPD620) Max (Normalized Counts, Total Weighting)",
                 "Nucleus CD20 (PPD620) Std Dev (Normalized Counts, Total Weighting)",
                 "Nucleus CD3 (PPD650) Mean (Normalized Counts, Total Weighting)",
                 "Nucleus CD3 (PPD650) Max (Normalized Counts, Total Weighting)",
                 "Nucleus CD3 (PPD650) Std Dev (Normalized Counts, Total Weighting)",
                 "Nucleus PANCK (PPD690) Mean (Normalized Counts, Total Weighting)",
                 "Nucleus PANCK (PPD690) Max (Normalized Counts, Total Weighting)",
                 "Nucleus PANCK (PPD690) Std Dev (Normalized Counts, Total Weighting)",
                 "Cytoplasm PD1 (PPD520) Mean (Normalized Counts, Total Weighting)",
                 "Cytoplasm PD1 (PPD520) Max (Normalized Counts, Total Weighting)",
                 "Cytoplasm PD1 (PPD520) Std Dev (Normalized Counts, Total Weighting)",
                 "Cytoplasm FOXP3 (PPD540) Mean (Normalized Counts, Total Weighting)",
                 "Cytoplasm FOXP3 (PPD540) Max (Normalized Counts, Total Weighting)",
                 "Cytoplasm FOXP3 (PPD540) Std Dev (Normalized Counts, Total Weighting)",
                 "Cytoplasm CD20 (PPD620) Mean (Normalized Counts, Total Weighting)",
                 "Cytoplasm CD20 (PPD620) Max (Normalized Counts, Total Weighting)",
                 "Cytoplasm CD20 (PPD620) Std Dev (Normalized Counts, Total Weighting)",
                 "Cytoplasm CD3 (PPD650) Mean (Normalized Counts, Total Weighting)",
                 "Cytoplasm CD3 (PPD650) Max (Normalized Counts, Total Weighting)",
                 "Cytoplasm CD3 (PPD650) Std Dev (Normalized Counts, Total Weighting)",
                 "Cytoplasm PANCK (PPD690) Mean (Normalized Counts, Total Weighting)",
                 "Cytoplasm PANCK (PPD690) Max (Normalized Counts, Total Weighting)",
                 "Cytoplasm PANCK (PPD690) Std Dev (Normalized Counts, Total Weighting)"]

In [ ]:
TOTAL_MAX = np.zeros(len(MXIF_FEATURES))
TOTAL_MIN = np.zeros(len(MXIF_FEATURES))

for i, feature in enumerate(MXIF_FEATURES):
    for core in train_mxIF:
        current_max = core.loc[:,feature].max()
        current_min = core.loc[:,feature].min()
        if current_max > TOTAL_MAX[i]:
            TOTAL_MAX[i] = current_max
        if current_min < TOTAL_MIN[i]:
            TOTAL_MIN[i] = current_min

In [ ]:
def get_generator(mxIF, cores):
    def data_generator():
        for i in range(len(mxIF)):
            X = mxIF[i].loc[:,'Cell X Position']
            Y = mxIF[i].loc[:,'Cell Y Position']

            for j,(x,y) in enumerate(zip(X, Y)):
                x = float(x)
                y = float(y)
                if np.isnan(x) or np.isnan(y):
                    continue
                if round(x - CELL_SIZE[0]) < 0 or round(x + CELL_SIZE[0]) >= cores[i].shape[1]:
                    continue
                if round(y - CELL_SIZE[1]) < 0 or round(y + CELL_SIZE[1]) >= cores[i].shape[0]:
                    continue

                cell_image = cores[i][round(y-CELL_SIZE[1]):round(y+CELL_SIZE[1]),
                                            round(x-CELL_SIZE[0]):round(x+CELL_SIZE[0])] / 255
                    
                cell_features = np.array(mxIF[i].loc[j, MXIF_FEATURES], dtype=np.float32)
                cell_features = (cell_features - TOTAL_MIN) / TOTAL_MAX
                    
                if np.sum(np.isnan(cell_features)) != 0:
                    continue

                yield (cell_image, cell_features)
                
    return data_generator

In [ ]:
def cell_coordinate_generator():
    for i in range(len(data_index)):
        X = data_mxIF[i].loc[:,'Cell X Position']
        Y = data_mxIF[i].loc[:,'Cell Y Position']

        for j,(x,y) in enumerate(zip(X, Y)):
            x = float(x)
            y = float(y)
            if np.isnan(x) or np.isnan(y):
                continue
            if round(x - CELL_SIZE[0]) < 0 or round(x + CELL_SIZE[0]) >= data_cores[i].shape[1]:
                continue
            if round(y - CELL_SIZE[1]) < 0 or round(y + CELL_SIZE[1]) >= data_cores[i].shape[0]:
                continue

            yield (x, y), i

In [ ]:
data = tf.data.Dataset.from_generator(get_generator(data_mxIF, data_cores),
                                       output_signature=(tf.TensorSpec(shape=(2*CELL_SIZE[1],2*CELL_SIZE[0],3), dtype=tf.float32),
                                                          tf.TensorSpec(shape=(len(MXIF_FEATURES)), dtype=tf.float32)))
data = data.batch(BATCH)

In [ ]:
load_dir = "logs/autoencoder/baseline_big/ffn_dropout/20230314-220323/"
loaded_model = tf.saved_model.load(load_dir)

In [ ]:
# calculate the latent vectors of all the training data
data_latent = []
for i, elem in enumerate(data):
    latent_he = loaded_model.encoder_conv(elem[0])
    latent_mxIF = loaded_model.encoder_fnn(elem[1])
    latent_he = latent_he.numpy()
    latent_mxIF = latent_mxIF.numpy()
    data_latent.append(np.concatenate([latent_he, latent_mxIF], axis=1))
    if i % 500 == 0:
        print(i)

In [ ]:
# calculate K cluster centroids in the latent space
K = 10
vectors_latent = np.concatenate(data_latent, axis=0)
vectors_latent_w = normalize(vectors_latent)

clusterer = KMeans(n_clusters=K).fit(vectors_latent_w)
labels = clusterer.predict(vectors_latent_w)

In [ ]:
# define colors for the K clusters
colormap = ['#0000FF', '#8A2BE2', '#FF4040', '#8A360F', '#98F5FF', '#FF6103', '#7FFF00', '#EEE8CD', '#FFB90F', '#556B2F', '#EE1289']
colormap = [ImageColor.getcolor(color, "RGB") for color in colormap]

In [ ]:
# plot the clustered cells in thre original H&E image for visualization
core_inx = 3
src = data_cores[core_inx]
cells = cell_coordinate_generator()
for i, ((x, y), j) in enumerate(cells):
    if j == core_inx:
        cv.circle(src, (int(x),int(y)), radius=4, color=colormap[int(labels[i])], thickness=-1)
    if j > core_inx:
        break


cv.imwrite("cluster.png", src)